## API 参数研究

In [ ]:
import requests
import json
pd.set_option('display.max_rows',None)


# PHIA Netherlands
# DSM Netherlands

# all primary dr in netherlands & america

country = 'netherlands'
columns = '["name","description","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market","is_primary","exchange","country"]'
payload = '{"columns":'+ columns +',"filter":[{"left":"name","operation":"nempty"},{"left":"typespecs","operation":"has_none_of","right":"preferred"},{"left": "type", "operation": "nequal", "right": "fund"}],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,600],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"}}'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}


url = 'https://scanner.tradingview.com/' + country + '/scan'
r = requests.post(url, headers = headers, data = payload).text
df = pd.DataFrame(json.loads(r)['data'])['d']
df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
df['turnover_rate'] = (df['Value.Traded'] / df['market_cap_basic']).round(4) 
df[(df['is_primary'] == False) & (df['type'] == 'dr')]

## 完整代码

In [ ]:
import requests
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
'''
fvey = america,australia,canada,newzealand,uk
EUR = belgium,france,germany,italy,luxembourg,netherlands,portugal,spain,switzerland*
北欧 = iceland,denmark,finland,norway,sweden

拉美 = argentina,brazil,chile,colombia,mexico,peru,venezuela
中东 = bahrain,egypt,israel,kuwait,qatar,turkey,uae,ksa(Saudi Arabia)
非洲 = kenya,morocco,nigeria,tunisia,rsa(South Africa)
东盟 = indonesia,malaysia,philippines,singapore,thailand,vietnam
南亚 = india,pakistan,srilanka

第一岛链 = japan,korea,taiwan

china,hongkong

russia,lithuania,latvia,estonia,serbia,hungary,romania,poland,slovakia,cyprus,greece

'''

fvey = ['america','australia','canada','newzealand','uk']
eur = ['belgium','france','germany','italy','luxembourg','netherlands','portugal','spain','switzerland','cyprus','greece']
n_eur = ['iceland','denmark','finland','norway','sweden']
china = ['china','hongkong']
east_asia = ['japan','korea','taiwan']
latin_america = ['argentina','brazil','chile','colombia','mexico','peru','venezuela']
middle_east = ['bahrain','egypt','israel','kuwait','qatar','turkey','uae','ksa']
africa = ['kenya','morocco','nigeria','tunisia','rsa']
asan = ['indonesia','malaysia','philippines','singapore','thailand','vietnam']
indian = ['india','pakistan','srilanka']
east_eur = ['russia','lithuania','latvia','estonia','serbia','hungary','romania','poland','slovakia']


country_list = fvey + eur + n_eur + china + east_asia + latin_america + middle_east + africa + asan + indian + east_eur

def get_USD_forex_table():
    url = 'https://scanner.tradingview.com/forex/scan'
    payload = '{"columns":["currency_logoid","base_currency_logoid","name","description","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","bid","ask","high","low","Recommend.All"],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,2000],"sort":{"sortBy":"name","sortOrder":"asc","nullsFirst":false},"preset":"forex_rates_all"}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

    r = requests.post(url, headers = headers, data = payload).text
    forex_df = pd.DataFrame(json.loads(r)['data'])['d']
    forex_df = pd.DataFrame(forex_df.tolist(), index = forex_df.index)
    USD_forex_table = forex_df[(forex_df[12] == 'USD')][[2,3,7]]
    USD_forex_table['currency_id'] =  USD_forex_table[2].str.replace('USD','')
    USD_forex_table['name'] =  USD_forex_table[3].str.replace(' / U.S. DOLLAR','')
    USD_forex_table['forex_rate'] =  USD_forex_table[7]
    USD_forex_table = USD_forex_table.reset_index(inplace = False)[['currency_id','forex_rate']]
    USD_forex_table = USD_forex_table.append({'currency_id':'USD','forex_rate':1}, ignore_index=True)
    return USD_forex_table

def fetch_country_data(country_list):
    
    # columns = ["name","description","logoid","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","Recommend.All","volume","Value.Traded","market_cap_basic","fundamental_currency_code","price_earnings_ttm","earnings_per_share_basic_ttm","number_of_employees_fy","sector","industry","market"]
    columns = '["name","description","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market"]'
    payload = '{"columns":'+ columns +',"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,9000],"sort":{"sortBy":"name","sortOrder":"asc","nullsFirst":false},"preset":"all_stocks"}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    
    all_country_df = pd.DataFrame()
    for country in country_list:
#         print(country)
        url = 'https://scanner.tradingview.com/' + country + '/scan'
        r = requests.post(url, headers = headers, data = payload).text
        df = pd.DataFrame(json.loads(r)['data'])['d']
        df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
        df = df.merge(USD_forex_table,left_on='fundamental_currency_code', right_on = 'currency_id')
        df = df[~df['change'].isnull()]
        df['Value.Traded'] = (df['Value.Traded']* df['forex_rate']/1000000).fillna(0) 
        df = df[df['Value.Traded'] > 0]
        df['market_cap_USD'] = df['market_cap_basic'] * df['forex_rate']
        df['market_cap_USD'] = (df['market_cap_USD']/100000000).fillna(0) 
        df = df[df['market_cap_USD'] > 0]
        all_country_df = all_country_df.append(df)
    return all_country_df


USD_forex_table = get_USD_forex_table()
df = fetch_country_data(country_list)

# fig = px.treemap(df, 
#                  path=['market','sector'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  values='market_cap_USD', # 需要聚合的列名
#                  color='change', 
#                  custom_data=['change','name','market_cap_USD','close'],
#                  range_color = [-8, 8], # 色彩范围最大最小值
# #                  hover_data= {'涨跌幅':':.2',
# #                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
#                  color_continuous_scale=["seagreen",'lightgrey', "indianred"],
#                  height = 900,
#                 #  width = 1600,
#                  color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
#                 )
# fig.update_coloraxes(showscale=False)
# fig.update_traces(textposition='middle center', 
#                   textinfo='label', 
#                   textfont = dict(color='white'),
#                   texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
#                   hovertemplate= "%{label}<br>%{customdata[3]:.2f}  (%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"                  
# #                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
#                  ) 
# fig.show()
df.to_csv('world_stock.csv',index = False, encoding = 'utf-8')

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('world_stock.csv', encoding = 'utf-8')



def check_plate(market):
    if market in fvey:
        market = 'Five Eyes'
    elif market in (eur + n_eur + east_eur):
        market = 'Europe'
    elif market in china:
        market = 'China'
    elif market in east_asia:
        market = 'East Asia'
    elif market in latin_america :
        market = 'Latin'
    elif market in middle_east + africa:
        market = 'Middle East + Africa'
    elif market in indian:
        market = 'South Asia'
    elif market in asan:
        market = 'ASEAN'
    else:
        market = 'Rest'
    return market

fvey = ['america','australia','canada','newzealand','uk']
eur = ['belgium','france','germany','italy','luxembourg','netherlands','portugal','spain','switzerland','cyprus','greece']
n_eur = ['iceland','denmark','finland','norway','sweden']
china = ['china','hongkong']
east_asia = ['japan','korea','taiwan']
latin_america = ['argentina','brazil','chile','colombia','mexico','peru','venezuela']
middle_east = ['bahrain','egypt','israel','kuwait','qatar','turkey','uae','ksa']
africa = ['kenya','morocco','nigeria','tunisia','rsa']
asan = ['indonesia','malaysia','philippines','singapore','thailand','vietnam']
indian = ['india','pakistan','srilanka']
east_eur = ['russia','lithuania','latvia','estonia','serbia','hungary','romania','poland','slovakia']

df['plate'] = df['market'].apply(check_plate)

# df = df[df['plate'].isin(['ASEAN'])]
fig = px.treemap(df, 
                 path=['plate','fundamental_currency_code','market','sector'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  path=['plate','','sector','industry',],
                 values='market_cap_USD', # 需要聚合的列名
                 color='change', 
                 custom_data=['change','description','market_cap_USD','close','market'],
                 range_color = [-8, 8], # 色彩范围最大最小值
#                  hover_data= {'涨跌幅':':.2',
#                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{label}<br>%{customdata[0]:.2f}%<br>%{customdata[4]}<br>总市值=%{customdata[2]:d}亿"                  
#                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
                 ) 
fig.show()

## 分板块详情

In [ ]:
a_df = df[df['plate'].isin(['Five Eyes'])]
fig = px.treemap(a_df, 
                 path=['plate','market','sector','industry','description'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  path=['plate','','sector','industry',],
                 values='market_cap_USD', # 需要聚合的列名
                 color='change', 
                 custom_data=['change','name','market_cap_USD','close','market'],
                 range_color = [-8, 8], # 色彩范围最大最小值
#                  hover_data= {'涨跌幅':':.2',
#                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{customdata[1]}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{label}<br>%{customdata[0]:.2f}%<br>%{customdata[4]}<br>总市值=%{customdata[2]:d}亿"                  
#                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
                 ) 
fig.show()